Visit tensorflow HUB to know the existing models

In [1]:
import numpy as np
import os

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_hub as hub
import cv2


In [2]:
d_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
d_dir = tf.keras.utils.get_file("flower_photos", origin=d_url, cache_dir="/content/drive/MyDrive", untar=True)

In [3]:
import pathlib
d_dir = pathlib.Path(d_dir)
d_dir

PosixPath('/content/drive/MyDrive/datasets/flower_photos')

In [4]:
list(d_dir.glob("**/*.jpg"))[:5]

[PosixPath('/content/drive/MyDrive/datasets/flower_photos/roses/7345657862_689366e79a.jpg'),
 PosixPath('/content/drive/MyDrive/datasets/flower_photos/roses/7409458444_0bfc9a0682_n.jpg'),
 PosixPath('/content/drive/MyDrive/datasets/flower_photos/roses/9337528427_3d09b7012b.jpg'),
 PosixPath('/content/drive/MyDrive/datasets/flower_photos/roses/7551637034_55ae047756_n.jpg'),
 PosixPath('/content/drive/MyDrive/datasets/flower_photos/roses/5736328472_8f25e6f6e7.jpg')]

In [5]:
data_image_dict={
    "roses": list(d_dir.glob("roses/*")),
    "daisy": list(d_dir.glob("daisy/*")),
    "dandelion": list(d_dir.glob("dandelion/*")),
    "sunflowers": list(d_dir.glob("sunflowers/*")),
    "tulips": list(d_dir.glob("tulips/*")),
}

In [6]:
label={
    "roses": 0,
    "daisy": 1,
    "dandelion": 2,
    "sunflowers": 3,
    "tulips": 4,
}


In [7]:
img_count=sum(map(len, data_image_dict.values()))
img_count

3670

In [ ]:
IMAGE_SIZE=(224,224)
x,y=[],[]
for fn,im in data_image_dict.items():
  for i in im:
    img=cv2.imread(str(i))
    r_img=cv2.resize(img,IMAGE_SIZE)
    x.append(r_img)
    y.append(label[fn])

In [ ]:
x=np.array(x)
y=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test=tts(x,y,random_state=0)

In [ ]:
x_train=x_train/255
x_test=x_test/255

In [ ]:
plt.axis("off")
plt.imshow(x[0])

In [53]:
import tensorflow as tf
from PIL import Image
import numpy as np

# Load the MobileNetV2 model
model = tf.keras.applications.MobileNetV2(
    input_shape=(180, 180, 3),  # Define the input shape
    alpha=1.0,
    include_top=False,          # Set 'include_top=False' if you don't want the classification layers
    weights='imagenet',
    pooling='avg',              # You can use 'avg' or 'max' pooling for feature extraction
    classes=5,
    classifier_activation='softmax'
)

# Load and preprocess the image
size = (180, 180)
img = Image.open("/content/goldfish.jpg").resize(size)

# Convert the image to a NumPy array and normalize it
img = np.array(img) / 255.0

# Expand dimensions to match the expected input shape of (batch_size, height, width, channels)
img = np.expand_dims(img, axis=0)

# Make predictions
predictions = model.predict(img)

# Print the prediction results
pre=np.argmax(predictions)


<ipython-input-53-7f1b59570e67>:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = tf.keras.applications.MobileNetV2(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [56]:
pre

949

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the MobileNetV2 model without the top layer (include_top=False)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),  # Image size (224x224x3)
    include_top=False,          # Do not include the final dense layer
    weights='imagenet'          # Load weights pre-trained on ImageNet
)

# Freeze the base model so that its weights are not updated during training
base_model.trainable = False

# Add custom layers for the flower classification task
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # Add a pooling layer
    layers.Dense(128, activation='relu'),  # Add a fully connected layer
    layers.Dropout(0.5),  # Add dropout for regularization
    layers.Dense(5, activation='softmax')  # Output layer with 5 classes (for your flower dataset)
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use categorical cross-entropy for multi-class classification
              metrics=['accuracy'])



In [ ]:
# Train the model
history = model.fit(
    x_train,y_train,
    epochs=10,  # Set the number of epochs

)


In [62]:
y_train[:5]

array([2, 0, 1, 2, 4])